# 2-6 数学的な問題を解くコツ

数学、特に数論は計算機科学と深く、しばしば競技プログラミングの題材として選ばれる。このような問題はアドホックな方法で解くことが多いが、ここで使える基本的なアルゴリズムを紹介する。

## ユークリッドの互除法

最大公約数を求めるアルゴリズム。
解説は[こちら](https://www.naganomathblog.com/entry/2020/05/16/080000)

計算量は$O(log(max(a, b)))$

In [ ]:
def gcd(a, b):
    # ユークリッドの互除法
    if b == 0:
        return a
    return gcd(b, a % b)

## 拡張ユークリッドの互除法

ユークリッドの互除法を拡張することで、次のような問題も解くことができる

### 双六

双六があります。升目には整数が書かれています。

升目は前後に無限に続いており、0の升目がスタートで、1の升目がゴールなのですが、サイコロの目にはa、b, -a, -bの4つの整数しかありません。

そのため、a, bの値によってはゴールに辿り着けない場合があります。

4つの目をそれぞれ何回出せばゴールに辿り着けますか？

複数解が存在する場合は、どれか1つを出力しなさい。解がなければ-1を出力しなさい。

制約

-   $1 \leqq a, b \leqq 10^9$


この問題は、式で表すと、

$ax + by = 1となる整数x, yを求めよ$

となる。このときaとbの最大公約数が1のとき以外は解が存在しないことは明らかなので、以下となる。

-   $gcd(a, b) \neq 1$ →解は存在しない
-   $gcd(a, b) = 1$ →ユークリッドの互除法を拡張することで、解を求めることができる

実は$ax + by = gcd(a, b)$の整数解(x, y)は常に存在し、同じアルゴリズムで求めることができる。

($ax + by = c が整数解を持つ \Leftrightarrow cはgcd(a, b)の倍数である$)[証明](https://manabitimes.jp/math/674)

さて、

$bx' + (a \% b)y' = gcd(a,b)$

の整数解x', y'が求まっているとする。

$a = (a // b) \times b + a \% b$(a//b = a割るbの商)を式変形して、
 
$a \% b =  a - (a // b) \times b$

これを代入すると

$bx' + (a - a // b \times b)y' = gcd(a, b)$

これを式変形して

$ay' + b(x' - (a // b) \times y') = gcd(a, b)$

となる

b=0のとき左辺=aなので、

$a \times 1 + b \times 0 = a = gcd(a, b)$

となり、x' = 1, y' = 0となる。

$b \neq 0$のときは

再帰的にgcd(b, a % b)を求める

これらをプログラムで表現する。

In [1]:
def extgcd(a: int, b: int) -> tuple[int, int, int]:
    d = a
    if b == 0:
        x, y = 1, 0
    else:
        d, y, x = extgcd(b, a % b)
        # ややこしいが、ここで y = x', x = y'となっていることに注意
        # ay' + b(x' - (a // b) * y') = gcd(a, b)
        # -> y = x' = (x' - (a // b) * y') なので以下の式となる
        y -= (a // b) * x
    return d, x, y  # = d, x', y'


d, x, y = extgcd(4, 11)
if d == 1:
    print(f"x:{x}, y:{y}")
else:
    print(-1)

x:3, y:-1


ax + by = gcd(a, b)の解の大きさ

extgcdの計算量は、再帰の仕方をみればgcdの計算量と同じ。

この関数で求まるax + by = gcd(a, b)の解の大きさはどれくらいか。

実は、$ab \neq 0$のときは $|x| \leqq b, |y| \leqq a$であることがわかる。

これを帰納的に示す。

b = 0となる直前、
つまりa % b = 0のときは、x = 0, y = 1なので成り立っています。

d, x', y' = extgcd(b, a % b)を呼び出した後に、$|x'| \leqq b, |y'| ≤ a \% b$とする。

x, y = extgcd(a, b)のx, yはx=x', y = y' - (a // b) * x'なので、次のようになる。

$|x| = |x'| \leqq b, \\
|y| = |y' - (a // b)x'| \leqq |y'| + (a // b) \times |x'| \leqq a \% b + (a // b) \times b = a$

## 素数に関する基本的なアルゴリズム

素数は暗号などに使われるので数多くのアルゴリズムが存在するが、

プログラミングコンテストで主に使われるのはエラトステネスの篩と簡単な素数判定、素因数分解

### 素数判定

与えられた整数に対して、素数かどうか判定しなさい

制約

- $a \leqq n \leqq 10^9$

素数とはちょうど約数を2つ持つような整数のこと。

nの約数はn未満なので、2 ~ n-1 までの整数でn代わりきれるか試せば、判定できる。

ここでnが約数dを持つとすると、n/dもnの約数である。

$n = d \times n / d$なので$min(d, n / d) \leqq \sqrt{n}$であり、$2 ~ \sqrt{n}$までの整数を試すだけで十分ということがわかる。

同じ考え方で、素因数分解や約数の列挙も計算t料$O(\sqrt{n})$で行える。

より高速なアルゴリズム(フェルマーテスト、$\rho$法、数体ふるい法など)もあるが多くの場合はこれで十分


In [4]:
# 入力はすべて正とする

n = 295927


def is_prime(n):
    # 素数判定
    i = 2
    while i**2 <= n:
        if n % i == 0:
            return False
        i += 1
    return n != 1  # 1の場合は例外


def divisor(n):
    # 約数の列挙
    res: list[int] = []
    i = 1
    while i**2 <= n:
        if n % i == 0:
            res.append(i)
            if i != n // i:
                res.append(n // i)
        i += 1
    return res


def prime_factor(n):
    # 素因数分解
    res = {}
    i = 2
    while i**2 <= n:
        while n % i == 0:
            if not i in res:
                res[i] = 0
            res[i] += 1
            n //= i
        i += 1
    if n != 1:
        res[n] = 1
    return res


print(is_prime(n))

print(divisor(n))

print(prime_factor(n))

False
[1, 295927, 541, 547]
{541: 1, 547: 1}


### エラトステネスの篩

素数判定する整数が1つなら$O(\sqrt{n})$の方法で十分ですが、多くの整数を素数判定したい場合には、より効率的な方法が必要。

#### 素数の個数

与えられた整数nに対して、n以下の素数はいくつ存在しますか？

制約
- $n \leqq 10^6$

n以下の素数を列挙するには、エラトステネスの篩と呼ばれるユークリッドの互除法と同じくらい古いアルゴリズムがあります。

1. まず、2以上n以下の整数をすべて書き出す
2. その中の最小の数である2は素数
3. 表から2の倍数をすべて取り除く
4. 表に残った最小の数である3はそれ以下の数で割り切れなかったので、素数
5. 表から3の倍数をすべて取り除く

6. 表に残った最小の数をmとすると、mは素数です
7. 表からmの倍数をすべて取り除きます

このような操作を繰り返すことで、次々に素数を列挙していくことができます。

エラトステネスの節の計算量は、$O(n log log n)$程度であることが知られています。プログラミングコンテストで使うようなサイズに対しては、大体のの線形時間と考えて問題ないでしょう。

In [9]:
MAX_N = 10**6

prime: list[int] = []
is_prime: list[bool] = [False] * (MAX_N + 1)


def sieve(n: int):
    # まずすべての素数判定リストをTrueにする、その後素数でない数をFalseに上書きする
    for i in range(n + 1):
        is_prime[i] = True
    # 0, 1は素数ではない
    is_prime[0] = False
    is_prime[1] = False
    # 2~nを素数判定していく
    for i in range(2, n + 1):
        # iが素数場合(Trueの場合)
        if is_prime[i]:
            prime.append(i)  # 素数iをリストに追加

            # 素数iの整数倍(2倍以上)の数jのループ
            for j in range(i * 2, n + 1, i):
                # jは素数でないのでFalseに上書き
                is_prime[j] = False
    return print(f"{len(prime)}({prime})")


sieve(11)

5([2, 3, 5, 7, 11])


### 区間篩

#### 区間内の素数の個数

与えられた整数a，bに対して、区間［a,b)に素数はいくつ存在しますか。

制約

- $a < b \leqq 10^{12}$
- $b - a \leqq 10^6$

素数判定の項でもやったように、b未満の素数でない整数の最小の素因数はたかだか$\sqrt{b}$なので、$\sqrt{b}$以下の素数の表があれば、エラトステネスの篩を[a, b)に用いることができます。つまり、[2, $\sqrt{b}$)の表と[a, b)の表を別々に作っておいて、[2, $\sqrt{b}$)の表から素数が得られるたびに、その素数の倍数を[a, b)から除けば[a, b)にある素数が列挙できます。

In [1]:
from math import sqrt, ceil

MAX_L = 10**6  # b - aの取りうる範囲
MAX_SQRT_B = 10**6
is_prime = [True] * MAX_L  # [a, b)のリスト
is_prime_small = [False] * MAX_SQRT_B  # √bのリスト

# [a, b)の整数に対して篩を行う。is_prime[i - a] = True ⇔ iが素数
def segment_sieve(a: int, b: int):
    # is_prime_smallを0 ≦ i < √bの範囲ですべてTrueに設定する(使う分だけTrueにする)
    for i in range(ceil(sqrt(b))):
        is_prime_small[i] = True

    # is_primeを0 ≦ i < b -a の範囲ですべてTrueに設定する(使う分だけ)
    for i in range(b - a):
        is_prime[i] = True

    for i in range(2, ceil(sqrt(b))):
        # is_prime_smallを2 ≦ i < √bの範囲でTrue()=iが整数)の場合(0, 1は素数でないので無視)
        if is_prime_small[i]:
            # iの整数倍(2倍以上)の値jは素数でないので、is_prime_smallをFalseにする
            for j in range(2 * i, ceil(sqrt(b)), i):
                is_prime_small[j] = False  # [2,√b)の篩

            # iの整数倍{(a + i - 1) // i} 倍以上)は素数でないのでFalseにする
            # is_prime[j - a] はjが素数かどうかの情報を持つ
            # {(a + i - 1) // i} * i は、a以上かつiの整数倍である最小の数を表す
            # {((a - 1) + i) // i} * iのほうが理解しやすいと思われる

            # python的には以下で書いたほうがわかりやすいか
            # x, y = divmod(a, i)  # xは商, yは余り
            # for j in range(a if y == 0 else (x + 1) * i, b, i)
            for j in range((a + i - 1) // i * i, b, i):
                is_prime[j - a] = False  # [a, b)の篩


a, b = 22801763489, 22801787297
segment_sieve(a, b)
ans = sum(is_prime[: b - a])
print(ans)

1000


### 余りの計算

#### なぜ余りを計算するか

プログラミングコンテストにおいて計算結果が64bitに収まらない場合は、適当な数で余りを計算させます。これは、多倍長演算による言語間のハンデをなくすためです。例えば、JavaではBigIntegerという多倍長演算をサポートするクラスがありますが、C++やC言語では自分で実装する必要があります。また、掛け算などの実装によって計算量そのものが変わってしまうことがあるので、アルゴリズムそのものを評価しづらくなります。このような理由から、プログラミングコンテストでは余りの計算がしばしば登場します。

#### 基本的な計算

mで余りをとることを、「mでmodをとる」や「mを法とする」と言うことがあります。以下、mでmodをとるとします。簡単に記述するために、整数aとbのmで割った余りが等しいことをa≡b (mod m)と書くことにします。また、aをmで割った余りをa mod mと書きます。a mod mOSa mod msm-1となるようにとります。環境によっては、aが負の場合にa % mも負になるので、a % m + mと置き換えて0～m-1の範囲に
収まるようにします。簡単な計算により、a ≡ c(mod m), b ≡ d(mod m)とすると、

$a + b \equiv c + d (mod \quad m) \\
a - b \equiv c - d (mod \quad m) \\
a \times b \equiv c \times d (mod \quad m)$

が成り立つことが言えます。
実際の問題ではmが$10^9$くらいの大きさであることがあり、$a \times b$の計算など、余りは32bitの範囲に収まるのに、余りをとる前には32bitに収まらない場合があるので、オーバーフローしないように注意する必要があります。オーバーフローの危険性があるときは、最初からすべての変数を64bit整数にしておくという手もあります。
modの世界でも$+、-、\times$は自然に計算できますが、割り算に関しては注意が必要です。
例えば2 ≡ 8 (mod 6)ですが、2 / 2 = 14 = 8 / 2(mod 6)となります。$a \times c = b \times c(mod \quad m)$の場合、$(a - b) \times c$はmで割り切れます。
d = gcd(c, m)とすると、$(a - b) \times (c / d)$がm / dで割り切れることになり、c / dとm / dは互いに素なのでa - bがm / dで割り切れることになります。
よって、a ≡ b(mod m / gcd(m, c))となります。≡を使った式は=の式とほぼ同じように計算できるので便利ですが、この場合のように定義に戻ってa - bがmで割り切れると考えたほうがよい場合があります。

### べき乗を高速に計算する

べき乗(指数)を高速に計算するには、繰り返し二乗法がある

#### Carmichael Numbers (UVa No.10006)

任意の1 < x < nに対して、$x^n \equiv x (mod \quad n)$が成り立つような素数でない整数nをCarmichael Numberと言います。与えられたnに対して、それがCarmichael Numberかどうか判定しなさい。

制約
- 2 < n < 65000

($x^n \equiv x (mod \quad n)$というのはつまり、$x^nとx両方において、nで割った余りが等しい$ということ)

この問題では、$O(n)$(単純にxをn回かける)でべき乗を行うと確かめる数がn個あるので、計算量が$O(n^2)$になって間に合いません。
べき乗を高速に計算することを考えましょう。

$n=2^k$のとき(指数nが2のべき乗で表せるとき)は

$ x^n = ((x^2)^2)... $

となるので、2乗をk回行うことで簡単に求まります。これを念頭に置くと、nを2のべき乗の和で表すとよさそうです。

$ n = 2^{k_1} + 2^{k_2} + 2^{k_3} $ 

とすると、

$ x^n = x^{2^{k_1}}x^{2^{k_2}}x^{2^{k_3}} $ 

となります。$x^{2^i}$を順次求めながら計算していけばいいので、結局O(log n)でべき乗が求まることになります。
自分で適当な数で試してみると理解しやすいと思います。

$x^{22} = x^{16} \times x^4 \times x^2$

(22は二進数で10110)
二進数でa桁目が1のとき$x^{2^a}$で分解できる

[参考ページ](https://compro.tsutaj.com//archive/170926_binary.pdf)

手順を以下にまとめると

$x^n$を求めたいとき($3^{22}$とする)

1. 指数部を2進数として扱う
    - $22_{(10)}=10110_{(2)}$
2. 指数部のiビット目(i桁目)が立っている(1になっている)なら、対応する２のべき乗の値を答えに掛け合わせる
    - $3^{22} = 3^{2^{4}} \times 3^{2^{2}} \times 3^{2^{1}}$

In [ ]:
# べき乗計算を自力で実装する場合


def mod_pow(x, n, mod):
    # 基本modで割った余りについて考えれば良いため、ところどころ % modとなっている
    res = 1
    while n > 0:  # 指数nが1以上のときループする
        if n & 1:  # &はビット演算子のand(例 11010 and 1 = 0と1のandなので0)。2進数で一桁目が1のときTrue。
            res = (res * x) % mod  # 最下位ビットが立っているときにxを掛ける

        # 次のループで2進数の次の桁を計算するための準備を行う
        x = (x**2) % mod  # xを二乗
        n >>= 1  # >>はビット演算によるビットの右シフト。11010 → 1101
    return res


# 再帰関数版
def mod_pow2(x, n, mod):
    if n == 0:
        return 1  # 2^0 = 1

    res = mod_pow2(x**2 % mod, n // 2, mod)
    if n & 1:
        res = (res * x) % mod
    return res

In [11]:
# pythonではpow関数がすでにあるので利用する
# Carmichael数の定義から素数は除外

from math import sqrt, floor

# 素数判定O(√n)
def is_prime(n):
    for i in range(2, floor(sqrt(n)) + 1):
        if n % i == 0:
            return False
    return n != 1  # 1の場合は例外


def solve(n):
    # nが素数であれば、'No'を出力してプログラムを終了する
    if is_prime(n):
        print("No")
        return

    # 2 <= x < n の整数xについて確認すればよい。つまりx % n = xである領域。
    # n以上の数yの場合、y mod n = z (2 <= (z = x) < n)についての判定となり、無駄。
    for x in range(2, n):
        # x^n ≡ x(mod n) が成り立たないようなxが存在すれば、'No'を出力してプログラムを終了する
        test = pow(x, n, n)
        if pow(x, n, n) != x:
            print("No")
            return
    print("Yes")
    return


solve(17)
solve(561)
solve(4)

No
Yes
No
